In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import timedelta
import pandas as pd

# Add src directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store


In [2]:
# Use a fixed timestamp since April data is unavailable
current_date = pd.Timestamp("2025-04-01 00:00:00", tz="UTC")
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=29)

print(f"⏳ Fetching data from {fetch_data_from} to {fetch_data_to}")


⏳ Fetching data from 2025-03-03 00:00:00+00:00 to 2025-03-31 23:00:00+00:00


In [3]:
feature_store = get_feature_store()
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

# Retrieve data with some buffer days
ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)

# Keep only records within desired range
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data = ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)


2025-05-03 00:51:43,355 INFO: Initializing external client
2025-05-03 00:51:43,356 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-03 00:51:44,005 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.37s) 


In [4]:
from src.data_utils import transform_ts_data_info_features

features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)


Skipping location_id 99: Not enough data to create even one window.


In [5]:
from src.inference import load_model_from_registry, get_model_predictions

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

# Add current prediction hour
predictions["pickup_hour"] = current_date.ceil("h")
predictions.head()


2025-05-03 00:52:02,628 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 00:52:02,630 INFO: Initializing external client
2025-05-03 00:52:02,630 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 00:52:03,104 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683


Downloading: 0.000%|          | 0/316530 elapsed<00:00 remaining<?

,pickup_location_id,predicted_demand,pickup_hour
0,2,0.0,2025-04-01 00:00:00+00:00
1,2,0.0,2025-04-01 00:00:00+00:00
2,3,0.0,2025-04-01 00:00:00+00:00
3,3,0.0,2025-04-01 00:00:00+00:00
4,4,5.0,2025-04-01 00:00:00+00:00


In [6]:
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

feature_group.insert(predictions, write_options={"wait_for_job": False})


2025-05-03 00:52:09,186 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 00:52:09,191 INFO: Initializing external client
2025-05-03 00:52:09,191 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 00:52:09,804 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683


Uploading Dataframe: 100.00% |██████████| Rows 510/510 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214683/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)